In [ ]:
!pip install langchain tiktoken chromadb pymupdf sentence-transformers openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 30.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

In [ ]:
os.environ["OPENAI_API_KEY"] = "여러분들의 Key 값"

In [ ]:
urls = [
   "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf",
   "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
]

# 각 파일 다운로드
for url in urls:
   filename = url.split("/")[-1]  # URL에서 파일명 추출
   response = requests.get(url)

   with open(filename, "wb") as f:
       f.write(response.content)
   print(f"{filename} 다운로드 완료")

ict_japan_2024.pdf 다운로드 완료
ict_usa_2024.pdf 다운로드 완료


In [ ]:
# 임베딩 설정
embd = OpenAIEmbeddings()

In [ ]:
def create_pdf_retriever(
    pdf_path: str,  # PDF 파일 경로
    persist_directory: str,  # 벡터 스토어 저장 경로
    embedding_model: OpenAIEmbeddings,  # OpenAIEmbeddings 임베딩 모델
    chunk_size: int = 512,  # 청크 크기 기본값 512
    chunk_overlap: int = 0  # 청크 오버랩 크기 기본값 0
) -> Chroma.as_retriever:

    # PDF 파일로드
    loader = PyMuPDFLoader(pdf_path)
    data = loader.load()

    # 청킹
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    doc_splits = text_splitter.split_documents(data)

    # 벡터 스토어로 적재
    vectorstore = Chroma.from_documents(
        persist_directory=persist_directory,
        documents=doc_splits,
        embedding=embedding_model,
    )

    return vectorstore.as_retriever()

In [ ]:
# 일본 ICT 정책 데이터베이스 생성
retriever_japan = create_pdf_retriever(
    pdf_path="ict_japan_2024.pdf",
    persist_directory="db_ict_policy_japan_2024",
    embedding_model=embd
)

# 미국 ICT 정책 데이터베이스 생성
retriever_usa = create_pdf_retriever(
    pdf_path="ict_usa_2024.pdf",
    persist_directory="db_ict_policy_usa_2024",
    embedding_model=embd
)

In [ ]:
jp_engine = create_retriever_tool(
    retriever=retriever_japan,
    name="japan_ict",
    description="일본의 ICT 시장동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요.",
)

usa_engine = create_retriever_tool(
    retriever=retriever_usa,
    name="usa_ict",
    description="미국의 ICT 시장동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요.",
)

In [ ]:
tools = [jp_engine, usa_engine]

In [ ]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)
print('--프롬프트 끝--')

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
--프롬프트 끝--


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
template = '''다음 질문에 최선을 다해 답변하세요. 당신은 다음 도구들에 접근할 수 있습니다:

{tools}

다음 형식을 사용하세요:

Question: 답변해야 하는 입력 질문
Thought: 무엇을 할지 항상 생각하세요
Action: 취해야 할 행동, [{tool_names}] 중 하나여야 합니다. 리스트에 있는 도구 중 1개를 택하십시오.
Action Input: 행동에 대한 입력값
Observation: 행동의 결과
... (이 Thought/Action/Action Input/Observation의 과정이 N번 반복될 수 있습니다)
Thought: 이제 최종 답변을 알겠습니다
Final Answer: 원래 입력된 질문에 대한 최종 답변

## 추가적인 주의사항
- 반드시 [Thought -> Action -> Action Input format] 이 사이클의 순서를 준수하십시오. 항상 Action 전에는 Thought가 먼저 나와야 합니다.
- 최종 답변에는 최대한 많은 내용을 포함하십시오.
- 한 번의 검색으로 해결되지 않을 것 같다면 문제를 분할하여 푸는 것은 중요합니다.
- 정보가 취합되었다면 불필요하게 사이클을 반복하지 마십시오.
- 묻지 않은 정보를 찾으려고 도구를 사용하지 마십시오.

시작하세요!

Question: {input}
Thought: {agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
# GPT-4o로부터 llm 객체를 선언
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# prompt_react를 사용하면 영어 프롬프트. prompt를 사용하면 한글 프롬프트
react_agent = create_react_agent(llm, tools=tools, prompt=prompt)

react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

<ipython-input-13-d032370d235d>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0)


In [ ]:
result = react_agent_executor.invoke({"input": "한국과 미국의 ICT 기관 협력 사례"})



> Entering new AgentExecutor chain...
이 질문은 미국의 ICT 시장 동향과 관련이 있으므로, 미국의 ICT 기관과 한국의 협력 사례를 찾기 위해 usa_ict 도구를 사용해야 합니다.

Action: usa_ict
Action Input: "한국과 미국의 ICT 기관 협력 사례"한국-미국 ICT 기업 진출 사례
• 다양한 산업과 규모의 한국 기업들이 미국 시장 진출을 위해 노력하고 있음 
[표 10] 한국-미국 협력 현황
구분
날짜
내용
FTA 체결여부
(발효)
12.03
‣ 한-미 FTA 발효
19.01
‣ 한-미 FTA 개정 의정서 발효
22.03
‣ 한-미 FTA 10주년 기념해 정부·국회 대표단 방미 일정 진행
정부
23.01
‣ 경북도, 산타클라라 한인상공회의소와 '실리콘밸리 스타트업 아카데미 
경북 MOU' 개최
23.01
‣ 광주시, AI 스타트업 미국 진출 지원
23.01

15 
Ⅰ. ICT 국가 산업 현황
 6. 한국 협력 및 국내기업 진출사례
 한국-미국 FTA 체결 여부
• 2012년 3월 한-미 FTA 발효됨
• 2019년 1월 한-미 FTA 개정 의정서 발효, 투자자-국가분쟁해결제도(ISDS)와 수출기업에 부담이 
된 무역구제 절차 개선됨
• 2022년 한-미 FTA 10주년을 기념해 정부·국회 대표단이 워싱턴 D.C(Washington DC), 
미시간(Michigan), 뉴욕(New York) 등을 방문함
 한국-미국 ICT 기관 협력 사례
• 과기정통부와 미국 기관 간 ICT 협력 사례가 주목됨

24 
Ⅱ. ICT 이슈 Top10 
 ⑦미국, 기술 교류를 위한 국가 간 협력 활발
 싱가포르와의 전략적 기술 파트너십 강화
• 로렌스 웡(Lawrence Wong) 싱가포르 부총리의 워싱턴 D.C. 방문을 계기로, 미국과 싱가포르는 
미-싱가포르 중요 및 신흥 기술(CET) 대화를 시작함. 대화는 연구, 혁신 및 상업적 관계를 
강화하여 과학적 지식의 국경을 확장하고 번영을 촉진하는 것을 목표로 하며

In [ ]:
print('최종 답변:', result['output'])

최종 답변: 한국과 미국의 ICT 기관 협력 사례로는 한-미 FTA 체결과 개정, 경북도와 산타클라라 한인상공회의소의 MOU 체결, 광주시의 AI 스타트업 미국 진출 지원 등이 있습니다. 또한, 과기정통부와 미국 기관 간의 ICT 협력도 주목받고 있습니다. 이러한 협력은 양국 간의 기술 교류와 상업적 관계를 강화하는 데 기여하고 있습니다.


In [ ]:
# 멀티 쿼리
result = react_agent_executor.invoke({"input": "미국과 일본의 ICT 주요 정책의 공통점과 차이점을 설명해줘."})



> Entering new AgentExecutor chain...
미국과 일본의 ICT 주요 정책의 공통점과 차이점을 설명하기 위해서는 두 나라의 ICT 정책에 대한 정보를 각각 수집해야 합니다. 이를 위해 각 나라의 ICT 시장동향 정보를 제공하는 도구를 사용하겠습니다.

Action: usa_ict
Action Input: 미국의 ICT 주요 정책에 대한 정보Ⅰ ICT 국가산업현황 
 4
(*) SUMMARY
1. 국가 개황
2. ICT 정부기구
3. ICT 주요정책
4. ICT 주요법령및규제
5. ICT 주요기업
6. 한국 협력 및 국내기업 진출사례
Ⅱ ICT 이슈Top 10
 16
(*) SUMMARY
① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력
② 미국, 일본과 양자컴퓨팅 개발 협력
③ 미국, 우주 클라우드 컴퓨팅 시장 주도
④ 미국, 드론 배송 도입 활발
⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발
⑥ 미국, 반도체 산업 활성화에 박차

23 
Ⅱ. ICT 이슈 Top10
 ⑥미국, 반도체 산업 활성화에 박차
 CHIPS 법안을 통한 연구개발 및 제조 지원 강화
• 미국 정부는 반도체법(CHIPS Act)에 따라 반도체 산업을 활성화하기 위해 R&D와 제조 지원에 
520억 달러(약 64조 8,400억 원)를 책정함. 이 조치는 반도체 생산 분야에서 미국의 경쟁력을 
회복하려는 목적이며, 자금 배분은 세계 공급량의 40%에서 10%로 급감한 국내 생산 능력을 
강화하기 위한 전략적 노력임

24 
Ⅱ. ICT 이슈 Top10 
 ⑦미국, 기술 교류를 위한 국가 간 협력 활발
 싱가포르와의 전략적 기술 파트너십 강화
• 로렌스 웡(Lawrence Wong) 싱가포르 부총리의 워싱턴 D.C. 방문을 계기로, 미국과 싱가포르는 
미-싱가포르 중요 및 신흥 기술(CET) 대화를 시작함. 대화는 연구, 혁신 및 상업적 관계를 
강화하여 과학적 지식의 국경을 확장하고 번영을 촉진하는 것을 목표로 하며, 인도-태평양 지역,

5 
Ⅰ. IC

In [ ]:
print('최종 답변:', result['output'])

최종 답변: 미국과 일본의 ICT 주요 정책의 공통점과 차이점은 다음과 같습니다.

공통점:
1. **기술 혁신과 산업 경쟁력 강화**: 두 나라는 모두 기술 혁신을 통해 산업 경쟁력을 강화하려는 노력을 하고 있습니다. 미국은 반도체 산업 활성화를 위해 CHIPS 법안을 통해 R&D와 제조 지원을 강화하고 있으며, 일본은 산업경쟁력 강화법을 개정하여 전략 분야의 국내 생산을 촉진하고 있습니다.
2. **국제 협력**: 미국과 일본 모두 국제 협력을 통해 기술 발전을 도모하고 있습니다. 미국은 싱가포르와의 전략적 기술 파트너십을 강화하고 있으며, 일본은 양자컴퓨팅 개발을 위해 미국과 협력하고 있습니다.

차이점:
1. **정책 초점**: 미국은 반도체 산업과 같은 특정 기술 분야에 집중하고 있으며, 일본은 Web3 산업 성장 촉진과 같은 새로운 기술 분야에 초점을 맞추고 있습니다.
2. **기술 도입**: 일본은 정부 행정 업무에 생성형 AI를 도입하는 등 공공 부문에서의 기술 활용을 강조하고 있으며, 미국은 의료 AI 연구와 같은 민간 부문에서의 기술 활용에 더 중점을 두고 있습니다.

이러한 공통점과 차이점은 두 나라가 각각의 경제적, 사회적 필요에 따라 ICT 정책을 조정하고 있음을 보여줍니다.


In [ ]:
result = react_agent_executor.invoke({"input": "미국의 ICT 관련 정부 기구, 주요 법령, 국내 기업 진출 사례 각각 따로 검색해. 그렇게 해서 정보 좀 모아봐. 그리고 나서 일본의 AI 정책도 알려줘."})



> Entering new AgentExecutor chain...
미국의 ICT 관련 정보를 수집하기 위해 각 항목에 대해 별도로 검색을 진행하겠습니다.

1. 미국의 ICT 관련 정부 기구
2. 미국의 ICT 관련 주요 법령
3. 미국의 ICT 관련 국내 기업 진출 사례

이후 일본의 AI 정책에 대해 알아보겠습니다.

먼저, 미국의 ICT 관련 정부 기구에 대해 알아보겠습니다.

Thought: 미국의 ICT 관련 정부 기구에 대한 정보를 수집해야 합니다.
Action: usa_ict
Action Input: "미국의 ICT 관련 정부 기구"
Ⅰ ICT 국가산업현황 
 4
(*) SUMMARY
1. 국가 개황
2. ICT 정부기구
3. ICT 주요정책
4. ICT 주요법령및규제
5. ICT 주요기업
6. 한국 협력 및 국내기업 진출사례
Ⅱ ICT 이슈Top 10
 16
(*) SUMMARY
① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력
② 미국, 일본과 양자컴퓨팅 개발 협력
③ 미국, 우주 클라우드 컴퓨팅 시장 주도
④ 미국, 드론 배송 도입 활발
⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발
⑥ 미국, 반도체 산업 활성화에 박차

27 
Ⅱ. ICT 이슈 Top10
 ⑩미 국방부, 디지털 트윈 기술 도입 확대
 국방부의 디지털 트윈 기술 활용 현황
• 디지털 트윈이 전투 수행에서의 변혁적 잠재력을 인정받고 있음. 지멘스 정부 기술 
(Siemens Government Technologies)의 존 우스티카(John Ustica) 사장은 광범위한 데이터 
입력을 활용하여 결과를 모델링하고 예측하는 기술의 능력을 강조함. 이러한 인식은 공급망 
관리 및 전장 능력 향상 등의 작업에서 디지털 트윈의 개발 및 적용을 가속화하기 위한

7 
Ⅰ. ICT 국가 산업 현황
 2. ICT 정부기구
  ② 연방통신위원회(FCC)
 미국 연방통신위원회(FCC)
• 미국 연방통신위원회(Federal Communications Commission)는 1934

In [ ]:
print('최종 답변:', result['output'])

최종 답변: 1. **미국의 ICT 관련 정부 기구:**
   - **연방통신위원회(FCC):** 1934년에 제정된 연방통신법에 의해 설립된 정보통신 및 방송 독립 행정 기관으로, 정보통신기술 및 방송에 관한 기본계획 수립 등 정책 총괄, 소유제한 등 시장 규제를 담당합니다.
   - **국가통신정보관리청(NTIA):** 상무부 산하의 기관으로, 통신 및 정보 정책 문제에 대해 대통령에게 자문을 제공하며, AI 관련 정책 권장 사항 개발을 위한 대중 참여 및 의견 요청 등을 시작했습니다.

2. **미국의 ICT 관련 주요 법령:**
   - **반도체 과학법(CHIPS and Science Act):** 2022년 7월에 승인된 법으로, 미국의 경쟁력을 강화하고 공급망을 탄력적으로 구축하여 국가 안보를 공고히 하며 주요 기술에 대한 접근을 지원하는 것을 목표로 합니다.

3. **미국의 ICT 관련 국내 기업 진출 사례:**
   - 다양한 한국 기업들이 미국 시장에 진출하고 있으며, 한-미 FTA 발효 및 개정 의정서 발효를 통해 협력 관계를 강화하고 있습니다. 예를 들어, 경북도와 산타클라라 한인상공회의소 간의 '실리콘밸리 스타트업 아카데미 경북 MOU' 개최, 광주시의 AI 스타트업 미국 진출 지원 등이 있습니다.

4. **일본의 AI 정책:**
   - 일본 경제산업성은 기업이 생성 AI 활용에 능숙한 인재를 육성하기 위한 지침을 마련하였으며, AI 및 데이터 분석 전문 인력 부족 문제를 해결하기 위해 AI의 적절한 활용을 장려하고 있습니다. 또한, 일본 개인정보보호위원회는 생성형 AI 서비스 사용에 관한 지침을 발표하여 기업과 일반 사용자가 유의해야 할 점을 제시하고 있습니다. 

이 정보들이 미국과 일본의 ICT 및 AI 정책에 대한 이해를 돕는 데 유용하길 바랍니다.
